In [3]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchsummary import summary
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:

args = {
    "data" : "data",
    "batch_size" : 64,
    "epochs" : 50,
    "lr" : 0.01,
    "momentum" : 0.5,
    "seed" : 1,
    "log_interval" : 10,
}

torch.manual_seed(args['seed'])

### Data Initialization and Loading
initialize_data(args["data"]) # extracts the zip files, makes a validation set

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args["data"] + '/train_values'),
    batch_size=args["batch_size"], shuffle=True, num_workers=1)
    val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args["data"] + '/val_images',
                         transform=data_transforms),
    batch_size=args["batch_size"], shuffle=False, num_workers=1)

### Neural Network and Optimizer
# We define neural net in model.py so that it can be reused by the evaluate.py script



In [ ]:
num_outputs = 2 # same for both CIFAR10 and MNIST, both have 10 classes as outputs

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
        
        

    def forward(self, input):
        input = input.view(-1, num_inputs) # reshape input to batch x num_inputs
        output = self.linear(input)
        return output

network = Net()
optimizer = optim.SGD(network.parameters(), lr=lr)

In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchsummary import summary
nclasses = 2 # GTSRB as 43 classes

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(3, 100 , kernel_size=3)
        self.pool1 = nn.MaxPool2d(2,2)
        self.dense_bn1 = nn.BatchNorm2d(100)
        self.conv2 = nn.Conv2d(100, 150 , kernel_size=4)
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(150 , 250, kernel_size=3)
        self.pool3 = nn.MaxPool2d(2,2)
        self.dense_bn2 = nn.BatchNorm2d(250)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(4000, 200)
        self.dense_bn = nn.BatchNorm1d(200)
        self.fc2 = nn.Linear(200, nclasses)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.dense_bn1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = self.dense_bn2(x)
        x = x.view(-1, 4000)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.dense_bn(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
summary(Net() , (3,48,48))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 100, 46, 46]           2,800
         MaxPool2d-2          [-1, 100, 23, 23]               0
       BatchNorm2d-3          [-1, 100, 23, 23]             200
            Conv2d-4          [-1, 150, 20, 20]         240,150
         MaxPool2d-5          [-1, 150, 10, 10]               0
            Conv2d-6            [-1, 250, 8, 8]         337,750
         MaxPool2d-7            [-1, 250, 4, 4]               0
       BatchNorm2d-8            [-1, 250, 4, 4]             500
            Linear-9                  [-1, 200]         800,200
      BatchNorm1d-10                  [-1, 200]             400
           Linear-11                    [-1, 2]             402
Total params: 1,382,402
Trainable params: 1,382,402
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forw

In [ ]:

def train(epoch):
    network.train()
    for batch_idx , (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        training_loss_arr.append(loss.data)
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))
    return training_loss_arr

def test():
    network.eval()
    test_loss = 0
    correct = 0
    test_loss_arr = []
    for data, target in test_loader:
        output = network(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').data # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    test_loss_arr.append(test_loss)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return test_loss_arr




In [7]:
# convlstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
 

In [8]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [10]:

 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + '/Users/agnithamohanram/Documents/Hack-NYU/data/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + '/Users/agnithamohanram/Documents/Hack-NYU/data/')
    print(testX.shape, testy.shape)
    # zero-offset class values
#     trainy = trainy - 1
#     testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy
 
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 0, 100, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_steps, n_length = 4, 32
    print("Cjhkjhjkhjk")
    print(trainX.shape[0])
    trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
#     testX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
#     testy = trainy
    testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    #model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    model.save('/Users/agnithamohanram/Documents/Hack-NYU/trained_model.hd5')
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy
 
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
    
#run an experiment
def run_experiment(repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)

    # summarize results
    summarize_results(scores)
 
    
# run the experiment
run_experiment(1)


(289, 128, 9) (289, 1)
(364, 128, 9) (364, 1)
(289, 128, 9) (289, 2) (364, 128, 9) (364, 2)
Cjhkjhjkhjk
289
>#1: 43.956
[43.956044005168664]
Accuracy: 43.956% (+/-0.000)
